# QSAR Analysis of Telomerase Inhibitors Part 2 - Exploratory Data Analysis

Import libraries

In [7]:
import numpy as np
import pandas as pd

from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

Load the bioactivity data

In [3]:
df = pd.read_csv("datasets/telomerase_03_curated.csv")
df

,molecule_chembl_id,canonical_smiles,standard_value,class
0,CHEMBL314057,CCN(CC)CC.O=C(N[C@H](Cc1c[nH]c2ccccc12)C(=O)NC...,7300.0,intermediate
1,CHEMBL266842,CC(C)(C)OC(=O)N[C@H](Cc1cc2ccccc2[nH]1)C(=O)N[...,100000.0,inactive
2,CHEMBL314847,CC(C)(C)OC(=O)N[C@H](Cc1c[nH]c2ccccc12)C(=O)NC...,100000.0,inactive
3,CHEMBL86984,CC(C)(C)OC(=O)N1CCC[C@@H]1C(=O)N[C@H](Cc1c[nH]...,9500.0,intermediate
4,CHEMBL87554,CCN(CC)CC.N[C@H](Cc1cc2ccccc2[nH]1)C(=O)N[C@H]...,100000.0,inactive
...,...,...,...,...
660,CHEMBL4632724,Cc1cn([C@H]2C[C@H](n3cc(COC(=O)c4ccc(S(N)(=O)=...,6000.0,intermediate
661,CHEMBL4637598,Cc1cn([C@H]2C[C@H](n3cc(CNC(=O)c4ccc(S(N)(=O)=...,9100.0,intermediate
662,CHEMBL4634543,Cc1cn([C@H]2C[C@H](n3cc(C[Se]c4ccc(NC(=S)Nc5cc...,5600.0,intermediate
663,CHEMBL4857212,O=C(OCCn1c([N+](=O)[O-])cnc1/C=C/c1ccc2ccccc2c...,980.0,active


In [4]:
df_no_smiles = df.drop(columns='canonical_smiles')

In [5]:
smiles = []

for i in df.canonical_smiles.tolist():
  cpd = str(i).split('.')
  cpd_longest = max(cpd, key = len)
  smiles.append(cpd_longest)

smiles = pd.Series(smiles, name = 'canonical_smiles')

In [6]:
df_clean_smiles = pd.concat([df_no_smiles,smiles], axis=1)
df_clean_smiles

,molecule_chembl_id,standard_value,class,canonical_smiles
0,CHEMBL314057,7300.0,intermediate,O=C(N[C@H](Cc1c[nH]c2ccccc12)C(=O)NCCCCCCCCCCC...
1,CHEMBL266842,100000.0,inactive,CC(C)(C)OC(=O)N[C@H](Cc1cc2ccccc2[nH]1)C(=O)N[...
2,CHEMBL314847,100000.0,inactive,CC(C)(C)OC(=O)N[C@H](Cc1c[nH]c2ccccc12)C(=O)NC...
3,CHEMBL86984,9500.0,intermediate,CC(C)(C)OC(=O)N1CCC[C@@H]1C(=O)N[C@H](Cc1c[nH]...
4,CHEMBL87554,100000.0,inactive,N[C@H](Cc1cc2ccccc2[nH]1)C(=O)N[C@H](Cc1c[nH]c...
...,...,...,...,...
660,CHEMBL4632724,6000.0,intermediate,Cc1cn([C@H]2C[C@H](n3cc(COC(=O)c4ccc(S(N)(=O)=...
661,CHEMBL4637598,9100.0,intermediate,Cc1cn([C@H]2C[C@H](n3cc(CNC(=O)c4ccc(S(N)(=O)=...
662,CHEMBL4634543,5600.0,intermediate,Cc1cn([C@H]2C[C@H](n3cc(C[Se]c4ccc(NC(=S)Nc5cc...
663,CHEMBL4857212,980.0,active,O=C(OCCn1c([N+](=O)[O-])cnc1/C=C/c1ccc2ccccc2c...


Create a function to calculate Lipinski descriptors

In [9]:
def lipinski(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i = 0  
    for mol in moldata:        
       
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
           
        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])   
    
        if(i == 0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i = i + 1      
    
    columnNames=["MW","LogP","NumHDonors","NumHAcceptors"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

In [10]:
df_lipinski = lipinski(df_clean_smiles.canonical_smiles)
df_lipinski

,MW,LogP,NumHDonors,NumHAcceptors
0,682.198,8.37560,4.0,5.0
1,922.417,6.35110,8.0,9.0
2,594.045,6.12970,4.0,6.0
3,775.324,8.46130,4.0,7.0
4,822.300,4.78510,8.0,8.0
...,...,...,...,...
660,506.497,-1.03798,3.0,12.0
661,505.513,-1.46498,4.0,11.0
662,691.654,-0.05638,5.0,11.0
663,464.481,5.52010,0.0,7.0


Combine the dataframe

In [12]:
df_combined = pd.concat([df,df_lipinski], axis=1)
df_combined

,molecule_chembl_id,canonical_smiles,standard_value,class,MW,LogP,NumHDonors,NumHAcceptors
0,CHEMBL314057,CCN(CC)CC.O=C(N[C@H](Cc1c[nH]c2ccccc12)C(=O)NC...,7300.0,intermediate,682.198,8.37560,4.0,5.0
1,CHEMBL266842,CC(C)(C)OC(=O)N[C@H](Cc1cc2ccccc2[nH]1)C(=O)N[...,100000.0,inactive,922.417,6.35110,8.0,9.0
2,CHEMBL314847,CC(C)(C)OC(=O)N[C@H](Cc1c[nH]c2ccccc12)C(=O)NC...,100000.0,inactive,594.045,6.12970,4.0,6.0
3,CHEMBL86984,CC(C)(C)OC(=O)N1CCC[C@@H]1C(=O)N[C@H](Cc1c[nH]...,9500.0,intermediate,775.324,8.46130,4.0,7.0
4,CHEMBL87554,CCN(CC)CC.N[C@H](Cc1cc2ccccc2[nH]1)C(=O)N[C@H]...,100000.0,inactive,822.300,4.78510,8.0,8.0
...,...,...,...,...,...,...,...,...
660,CHEMBL4632724,Cc1cn([C@H]2C[C@H](n3cc(COC(=O)c4ccc(S(N)(=O)=...,6000.0,intermediate,506.497,-1.03798,3.0,12.0
661,CHEMBL4637598,Cc1cn([C@H]2C[C@H](n3cc(CNC(=O)c4ccc(S(N)(=O)=...,9100.0,intermediate,505.513,-1.46498,4.0,11.0
662,CHEMBL4634543,Cc1cn([C@H]2C[C@H](n3cc(C[Se]c4ccc(NC(=S)Nc5cc...,5600.0,intermediate,691.654,-0.05638,5.0,11.0
663,CHEMBL4857212,O=C(OCCn1c([N+](=O)[O-])cnc1/C=C/c1ccc2ccccc2c...,980.0,active,464.481,5.52010,0.0,7.0


Convert IC50 to pIC50

pIC50 = -log10(IC50)

Why? to make IC50 data to be more uniformly distributed

In [13]:
def pIC50(input):
    pIC50 = []

    for i in input['standard_value_norm']:
        molar = i*(10**-9) # Converts nM to M
        pIC50.append(-np.log10(molar))

    input['pIC50'] = pIC50
    x = input.drop('standard_value_norm', 1)
        
    return x